The following example shows how to generate training data. We want to train the drone flying in y-z plane with different wall distance. 
Notes:
- Turn off disturbance observer to let the data label capture the entirety disturbance force. 
- Run the scenario in simulation_user_guide.ipynb first to make sure the drone does not hit the wall. Adjust wall distance from performance.
- Use the same trajectory in every condition as the same setting in Neural Fly paper.
- Use RandomWaypointsInConstrainedSpace to fly in a constrained 3D space in front of a wall.
- When training disturbance model with inflow model in controller, enable inflow model option in controller.  
- Use baseline disturbacne observer to generate training data.
- Ensure the disturbance model is set to the correct condition, i.e. setting of sinusoidal wind
- restart the ipynb before run


In [2]:
import sys
import os
import numpy as np

# Get the current directory of the notebook
notebook_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import manager
import drone.disturbance_model
import drone.trajectory

def extra_format(value):
    val = abs(value)
    s = f"{val:.1f}".replace('.', '_')
    return f"n{s}" if value < 0 else s

config_template = {
    "t_data_collection": None,
    "file_name_template": None,
    "wall_distance_range": None,
    "wind_range_x": None,
    "wind_range_y": None,
    "wind_range_z": None,
    "is_csv": False,
    "trajectory": None,
}

training_trajectory = drone.trajectory.RandomWaypointsInConstrainedSpace(200, False)
validation_trajectory = drone.trajectory.Figure8(20)


training_config = config_template.copy()
training_config["t_data_collection"] = 60.0
training_config["file_name_template"] = "test_wind_near_wall_x{}_y{}_z{}_d{}_train_no_bemt"
training_config["wall_distance_range"] = [0.1]  # in a constrained space, the wall distance naturally varies, so there is no need to set multiple values
training_config["wind_range_x"] = [-3.0, -1.0, 0.0]
training_config["wind_range_y"] = [0.0]
training_config["wind_range_z"] = [-5.0, -3.0, 0.0, 3.0, 5.0]
training_config["is_csv"] = True
training_config["trajectory"] = training_trajectory

validation_config = config_template.copy()
validation_config["t_data_collection"] = 40.0
validation_config["file_name_template"] = "test_wind_near_wall_x{}_y{}_z{}_d{}_validation_no_bemt"
validation_config["wall_distance_range"] = [0.1]  # in a constrained space, the wall distance naturally varies, so there is no need to set multiple values
validation_config["wind_range_x"] = [-5.0, -2.0, 0.0]
validation_config["wind_range_y"] = [0.0]
validation_config["wind_range_z"] = [-10.0, -4.0, -1.0, 1.0, 4.0, 10.0]
validation_config["is_csv"] = True
validation_config["trajectory"] = validation_trajectory

fitting_config = config_template.copy()
fitting_config["t_data_collection"] = 15.0
fitting_config["file_name_template"] = "test_wind_near_wall_x{}_y{}_z{}_d{}_fitting"
fitting_config["wall_distance_range"] = [100.0]  # in case need to treat as no wall
fitting_config["wind_range_x"] = [-3.0, 0.0]
fitting_config["wind_range_y"] = [0.0]
fitting_config["wind_range_z"] = [-5.0, 0.0, 5.0]
fitting_config["is_csv"] = False
fitting_config["trajectory"] = training_trajectory

fitting_validation_config = config_template.copy()
fitting_validation_config["t_data_collection"] = 40.0
fitting_validation_config["file_name_template"] = "test_wind_near_wall_x{}_y{}_z{}_d{}_validation"
fitting_validation_config["wall_distance_range"] = [0.1]  # in a constrained space, the wall distance naturally varies, so there is no need to set multiple values
fitting_validation_config["wind_range_x"] = [-5.0, -2.0, 0.0]
fitting_validation_config["wind_range_y"] = [0.0]
fitting_validation_config["wind_range_z"] = [-10.0, -4.0, -1.0, 1.0, 4.0, 10.0]
fitting_validation_config["is_csv"] = False
fitting_validation_config["trajectory"] = validation_trajectory

selected_config = fitting_config  # Change this to training_config, validation_config, fitting_config, or fitting_validation_config as needed


x_position_error = 0.4  # accounting for controller error to prevent collision to the wall
wall_distance_range = [distance + x_position_error for distance in selected_config["wall_distance_range"]]

is_csv = False  # save as csv file
for wall_distance in wall_distance_range:
    for wind_speed_x in selected_config["wind_range_x"]:
        for wind_speed_y in selected_config["wind_range_y"]:
            for wind_speed_z in selected_config["wind_range_z"]:
                print(f"Running simulation with wind speed ({wind_speed_x}, {wind_speed_y}, {wind_speed_z}) and wall distance {wall_distance}")
                # Initialize the simulation manager
                sim_test = manager.Manager()
                model = drone.disturbance_model.WindEffectNearWall(wall_origin=np.array([-wall_distance, 0, 0]), 
                                                                u_free = np.array([wind_speed_x, wind_speed_y, wind_speed_z]))
                # model = drone.disturbance_model.Free()    # debug only
                # trajectory=drone.trajectory.RandomWaypointsInConstrainedSpace(200, False)
                trajectory = selected_config["trajectory"]
                sim_test.set_up(disturbance_model=model, trajectory=trajectory)
                sim_test.run(selected_config["t_data_collection"]) 
                
                # Save the result
                file_name = selected_config["file_name_template"].format(extra_format(wind_speed_x), 
                                                    extra_format(wind_speed_y),
                                                    extra_format(wind_speed_z),
                                                    extra_format(wall_distance))
                if is_csv:
                    sim_test.save_result_as_csv(file_name)
                else:
                    sim_test.save_result_as_pkl(file_name)



Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Map data read from: ..\data\map\figure_8.pkl
Running simulation with wind speed (-3.0, 0.0, -5.0) and wall distance 100.4
[PropellerLookupTable] 

Optionally, save the header index into a file for future reference.

In [ ]:
sim_test.logger.generate_column_map()